In [38]:
import carto2gpd
import cenpy
import numpy as np
import pandas as pd
import dask.dataframe as dd
import geopandas as gpd
import datetime as dt
import missingno as msno

import hvplot.pandas
import holoviews as hv

from io import BytesIO
import gzip
from urllib.request import urlopen

In [43]:
states = ['pa','il','ga','ca']
years = ['2009','2010','2011','2012','2013','2014','2015','2016','2017','2018','2019']

In [26]:
url = "https://lehd.ces.census.gov/data/lodes/LODES7/pa/od/pa_od_aux_JT00_"
od = dd.concat([pd.read_csv(gzip.open(BytesIO(urlopen(url+year+".csv.gz").read()))) for year in years])

In [36]:
acs = cenpy.remote.APIConnection("ACSDT5Y2019").set_mapservice("tigerWMS_ACS2019")
phl_bg = acs.mapservice.layers[10].query(where="STATE = '42' and COUNTY = '101'")

/Users/bensh/.pyenv/versions/miniconda3-latest/envs/musa-550-fall-2021/lib/python3.8/site-packages/pyproj/crs/crs.py:68: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [42]:
phl_bg.hvplot(geo=True, 
                crs=3857, 
                legend=True,
                width=600, 
                height=400, 
                cmap='viridis')

:Polygons   [Longitude,Latitude]